<a href="https://colab.research.google.com/github/Llmortimer/character_morality/blob/main/DissertationFilmAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
!apt-get install -y ffmpeg

import re
import string
import numpy as np
from difflib import SequenceMatcher
from operator import itemgetter
from types import CoroutineType


input_script = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeScript.txt"
input_subtitles = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeSubtitles.txt"
dialogue_script = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeDialogue.txt"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [24]:
#pre-process subtitles by extractings start and end times of each dialogue.
f = open(input_subtitles).read()
matches = re.findall(r"([0-9]*)\n(.+)--> (.+)\n((.+\n{0,1})+)", f, re.M)
id = []
start = []
end = []
dialogue_sub = []
for i in matches:
    id.append(i[0])
    start.append(i[1])
    end.append(i[2])
    dialogue_sub.append(i[3].rstrip().replace('\n', ' '))

In [25]:
#pre-process script by extracting the dialogue and character names writing to seperate file.
f = open(input_script).read()
matches = re.findall(r"\n\s+([A-Z ]+)\n *((.+\s)*)", f, re.M)
dialogue = []
characters = []
for i in matches:
    if '' not in i:
        j = " ".join(i[1].split())
        dialogue.append(j)
        characters.append(i[0])

        

with open(dialogue_script, "w") as ds:
    for inst in dialogue:
        ds.write(inst+"\n")

In [26]:
print(len(dialogue))
print(len(dialogue_sub))

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

combined = []
last = 0

for c, i in enumerate(dialogue_sub):
    for count, j in enumerate(dialogue):

        if similar(i, j) > 0.7 and count > last and count < last + 10:
            combined.append((similar(i, j), c, count, i, j))
            last = count
            break

list1 = []
list2 = []

last_sub = -1
last_script = -1

for c in combined:
    if c[1] > last_sub+1:
        list1.append(list(range(last_sub+1, c[1])))
    elif c[1] == last_sub+1:
        list1.append([])

    list1.append([c[1]])
    last_sub = c[1]

    if c[2] > last_script+1:
        list2.append(list(range(last_script+1, c[2])))
    elif c[2] == last_script+1:
        list2.append([])

    list2.append([c[2]])
    last_script = c[2]

list1.append(list(range(last_sub+1, len(dialogue_sub))))
list2.append(list(range(last_script+1, len(dialogue))))

812
1431


In [ ]:
for c in combined:
    print(c)

In [27]:
char_annotated = []

for sbt, sct in zip(list1, list2):
    char = []
    dia = []
    diaSbt = []
    for i in sct:
        char.append(characters[i])
        dia.append(dialogue[i])
    
    for i in sbt:
        diaSbt.append(dialogue_sub[i])

    if len(sbt) != 0 and len(sct) != 0: 
        char_annotated.append([char, sbt, sct, diaSbt, dia, start[sbt[0]], end[sbt[-1]]])
    elif len(sbt) == 0 and len(sct) != 0:
        char_annotated.append([char, sbt, sct, ['no subtitle dialogue'], dia, 'no time info', 'no time info'])
    elif len(sct) == 0 and len(sbt) != 0 :
        char_annotated.append([['null'], sbt, sct, diaSbt, ['no script dialogue'], start[sbt[0]], end[sbt[-1]]])
    

for i in char_annotated:
    print(i)

[['OFFICER'], [0], [0], ['Dismount!'], ['Dis-mount!'], '00:01:57,283 ', '00:01:58,716']
[['FIRST SCOUT'], [1], [1], ["Herman's horse-sick!"], ["Herman's horsesick!"], '00:02:04,991 ', '00:02:07,425']
[['null'], [2], [], ['Chaps, no one wander off.'], ['no script dialogue'], '00:02:14,067 ', '00:02:17,059']
[['SCOUTMASTER'], [3], [2], ['Some of the passageways in here can run for miles.'], ["Chaps, don't anybody wander off. Some of the passageways in here can run for miles."], '00:02:17,137 ', '00:02:20,300']
[['HERMAN'], [4], [3], ["I don't think this is such a good idea."], ["I don't think this is such a good idea."], '00:02:30,617 ', '00:02:33,711']
[['HERMAN'], [5], [4], ['What is it?'], ['What is it?'], '00:02:37,190 ', '00:02:38,885']
[['FEDORA'], [6], [5], ['Alfred, did you get anything yet?'], ['Alfred, did you get anything yet?'], '00:03:04,050 ', '00:03:05,642']
[['MAN'], [7, 8], [6], ['Nothing.', 'Then keep digging.'], ['Nothing. Dig in.'], '00:03:05,718 ', '00:03:07,708']
[[

In [ ]:
# #for each subtitles dialogue without script dialogue and character check dialogue before and after
# #as it may be a substrings of before/after 

# for count, dialogue in enumerate(char_annotated):
#     if dialogue[0] == ['null']:
#         query_dialogues = dialogue[3]
#         for i, query_dialogue in enumerate(query_dialogues):
#             if query_dialogue in char_annotated[count-1][4][0] and query_dialogue not in char_annotated[count-1][3][0]:
#                 #if its present in the previous script dialogue but not in the subtitles dialogue: add it
#                 print('prev')
#                 char_annotated[count-1][3].append(query_dialogue)
#                 char_annotated[count-1][1].append(dialogue[1][i])
#                 char_annotated[count-1][5] = dialogue[5]
#                 char_annotated[count][3].remove(query_dialogue)
#             elif query_dialogue in char_annotated[count+1][4][0] and  query_dialogue not in char_annotated[count+1][3][0]:
#                 #if it is present in the next script dialogue but not in the subtitles dialogue: add it
#                 print('post')
#                 char_annotated[count+1][3].insert(0, query_dialogue)
#                 char_annotated[count+1][1].insert(0, dialogue[1][i])
#                 char_annotated[count+1][5] = dialogue[5]
#                 char_annotated[count][3].remove(query_dialogue)
    
#         print(len(char_annotated[count][3]))
#         print(len(char_annotated[count][4]))
#         if len(char_annotated[count][3]) == 0 and len(char_annotated[count][4]) == 0:
#             print("remove")
#             char_annotated.remove(dialogue)

            

In [28]:
edits = []

for count, dialogue in enumerate(char_annotated):
    #where the speaker of the dialogue is set to 'null'
    if dialogue[0] == ['null']:
        #we only consider the subtitle dialogues to move to other peices of dialogue as monologues exist within the script
        query_dialogues = dialogue[3]
        #for every line in the script dialogues
        for i, query_dialogue in enumerate(query_dialogues):
            #check if it is present in the previous or the post dialogues
            if query_dialogue.translate(str.maketrans('', '', string.punctuation)) in char_annotated[count-1][4][0].translate(str.maketrans('', '', string.punctuation)) and query_dialogue not in char_annotated[count-1][3][0]:
                #if its present in the previous script dialogue but not in the subtitles dialogue: add details to a list of edits to do
                edits.append((count, count-1, dialogue[1][i], len(query_dialogues)-i, query_dialogue))

            elif query_dialogue in char_annotated[count+1][4][0] and  query_dialogue not in char_annotated[count+1][3][0]:
                #if it is present in the next script dialogue but not in the subtitles dialogue: add details to a list of edits to do
                edits.append((count, count+1, dialogue[1][i], len(query_dialogues)-i, query_dialogue))


In [29]:
for c in edits:
    print(c)

(19, 20, 31, 1, 'Run back and find the others.')
(33, 34, 52, 1, 'Hey!')
(47, 48, 75, 1, 'Hold it.')
(64, 63, 91, 1, "illuminate me.''")
(74, 75, 105, 1, 'You lost today, kid,')
(78, 79, 109, 1, 'This is the second time')
(85, 84, 126, 2, 'in the library.')
(85, 84, 127, 1, 'Research. Reading.')
(93, 94, 142, 2, 'Well done, Indy.')
(93, 94, 143, 1, 'Very well done, indeed.')
(95, 96, 145, 1, 'We can discuss my honorarium')
(98, 99, 148, 1, 'Yes.')
(122, 123, 218, 1, "Hard to resist, isn't it?")
(130, 129, 263, 1, 'from underneath your feet.')
(148, 149, 309, 1, 'The search for the Cup of Christ')
(162, 161, 336, 1, 'Yes?')
(193, 192, 384, 2, 'You see?')
(193, 194, 385, 1, 'Look, Indy.')
(231, 232, 446, 1, 'Look at the artistry')
(239, 240, 459, 1, "Wouldn't it be wonderful")
(259, 258, 484, 1, 'Did you kill him too?')
(292, 293, 558, 1, 'You had it?')
(296, 297, 562, 1, 'Oh, yes.')
(310, 311, 585, 1, "Don't know.")
(325, 324, 613, 1, "I'll be right back.")
(346, 347, 643, 1, 'I knew it

In [30]:
#in the list edits there contains: (index of line in char_annotated to be moved, index of what line it should be in, exact id of the line to be moved, if it is moved forwards or backwards, the line to be moved)
for e in edits:
    if e[0] > e[1]:
        #append the literal line to the previous one 
        char_annotated[e[1]][3].append(e[4])
        #edit the end time of the appended line
        char_annotated[e[1]][6] = end[e[2]]
        #append the id of the line to the previous one
        char_annotated[e[1]][1].append(e[2])
    else:
        #insert the literal line to the beginning previous one 
        char_annotated[e[1]][3].insert(0, e[4])
        #edit the end time of the inserted line
        char_annotated[e[1]][6] = start[e[2]]
        #insert the id of the line to the previous one
        char_annotated[e[1]][1].insert(0, e[2])

i = 0
for e in edits:
    if e[3] == 1:
        del char_annotated[e[0]-i]
        i+=1

In [35]:
for i in char_annotated:
    print(i)

[['OFFICER'], [0], [0], ['Dismount!'], ['Dis-mount!'], '00:01:57,283 ', '00:01:58,716']
[['FIRST SCOUT'], [1], [1], ["Herman's horse-sick!"], ["Herman's horsesick!"], '00:02:04,991 ', '00:02:07,425']
[['SCOUTMASTER'], [3], [2], ['Some of the passageways in here can run for miles.'], ["Chaps, don't anybody wander off. Some of the passageways in here can run for miles."], '00:02:17,137 ', '00:02:20,300']
[['HERMAN'], [4], [3], ["I don't think this is such a good idea."], ["I don't think this is such a good idea."], '00:02:30,617 ', '00:02:33,711']
[['HERMAN'], [5], [4], ['What is it?'], ['What is it?'], '00:02:37,190 ', '00:02:38,885']
[['FEDORA'], [6], [5], ['Alfred, did you get anything yet?'], ['Alfred, did you get anything yet?'], '00:03:04,050 ', '00:03:05,642']
[['MAN'], [7, 8], [6], ['Nothing.', 'Then keep digging.'], ['Nothing. Dig in.'], '00:03:05,718 ', '00:03:07,708']
[['ROSCOE'], [9], [7], ["The kid's got something!"], ["Hey, I've got something!"], '00:03:07,787 ', '00:03:08,

In [34]:
to_delete = []

for count, i in enumerate(char_annotated):
    if i[0] == ['null']:
        to_delete.append(count)

    if i[5] == 'no time info' and i[6] == 'no time info':
        to_delete.append(count)

for count, i in enumerate(to_delete):
    del char_annotated[i-count]

In [42]:
final_dialogue = []

for i in char_annotated:
    for count, j in enumerate(i[0]):
        temp, ms = i[5].split(',')
        h, m, s = temp.split(':')
        start_time = int(h) * 3600 + int(m) * 60 + int(s) + float(ms)/1000
        temp, ms = i[6].split(',')
        h, m, s = temp.split(':')
        end_time = int(h) * 3600 + int(m) * 60 + int(s) + float(ms)/1000

        final_dialogue.append([j, i[4][count], start_time, end_time])

for i in final_dialogue:
    print(i)

['OFFICER', 'Dis-mount!', 117.283, 118.716]
['FIRST SCOUT', "Herman's horsesick!", 124.991, 127.425]
['SCOUTMASTER', "Chaps, don't anybody wander off. Some of the passageways in here can run for miles.", 137.137, 140.3]
['HERMAN', "I don't think this is such a good idea.", 150.617, 153.711]
['HERMAN', 'What is it?', 157.19, 158.885]
['FEDORA', 'Alfred, did you get anything yet?', 184.05, 185.642]
['MAN', 'Nothing. Dig in.', 185.718, 187.708]
['ROSCOE', "Hey, I've got something!", 187.787, 188.754]
['ROSCOE', 'I got something, Garth!', 191.558, 192.547]
['ROSCOE', 'I got something... I got something right here.', 192.625, 195.185]
['ROSCOE', "(more whoops; then) Oh, look at that! (more whoops) We're rich! We're rich!", 201.801, 205.571]
['HALFBREED', 'Shut up. Shut up.', 205.638, 207.435]
['HERMAN', 'Indy? Indy? What are they doing? Indiana? Indiana?', 207.507, 219.107]
['INDY', 'Shhh!', 219.185, 221.016]
['HALFBREED', 'Hey, we got to find more stuff to bring back.', 224.257, 226.35]
['

In [ ]:
#scene detection, for each frame assign a probablility that the scene has changed given the change of intensity of the pixels on the pervious frame.
#if this probablility is higher than 0.5 for a given frame the frame time, and probablilty have been output to a file "time.txt" and the frame itself output to a PNG in scenes/ 

!ffmpeg -i /content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeFilm.mp4 -filter_complex "select='gt(scene,0.5)',metadata=print:file=/content/drive/MyDrive/shared/scenes/time.txt" -vsync vfr /content/drive/MyDrive/shared/scenes/img%03d.png

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [44]:
#extract the scene changes and their times from the text file generated by the scene change algorithm

f = open('/content/drive/MyDrive/shared/scenes/time.txt').read()
matches = re.findall(r"frame:([0-9]+).+pts_time:([0-9.]+)", f)

scenes = {}
previous = '0'

for match in matches:
    scenes[match[0]] = {}
    scenes[match[0]]['start'] = float(previous)
    scenes[match[0]]['end'] = float(match[1])
    previous = match[1]

In [33]:
print(scenes)

{'0': {'start': '0', 'end': '47.1722'}, '1': {'start': '47.1722', 'end': '58.8922'}, '2': {'start': '58.8922', 'end': '66.9002'}, '3': {'start': '66.9002', 'end': '79.5796'}, '4': {'start': '79.5796', 'end': '88.9223'}, '5': {'start': '88.9223', 'end': '102.186'}, '6': {'start': '102.186', 'end': '118.785'}, '7': {'start': '118.785', 'end': '122.831'}, '8': {'start': '122.831', 'end': '129.338'}, '9': {'start': '129.338', 'end': '140.39'}, '10': {'start': '140.39', 'end': '147.94'}, '11': {'start': '147.94', 'end': '287.246'}, '12': {'start': '287.246', 'end': '301.635'}, '13': {'start': '301.635', 'end': '308.851'}, '14': {'start': '308.851', 'end': '311.52'}, '15': {'start': '311.52', 'end': '313.438'}, '16': {'start': '313.438', 'end': '316.733'}, '17': {'start': '316.733', 'end': '331.415'}, '18': {'start': '331.415', 'end': '336.962'}, '19': {'start': '336.962', 'end': '357.274'}, '20': {'start': '357.274', 'end': '361.069'}, '21': {'start': '361.069', 'end': '362.446'}, '22': {'s

In [53]:
scene_dialogues = []
character_list = []

for s in scenes:
    for d in final_dialogue:
        if d[2] > scenes[s]['start'] and d[3] < scenes[s]['end']:
            scene_dialogues.append((s, d))

for i in scene_dialogues:
    print(i)

for i, j in scene_dialogues:
    if j[0] not in character_list:
        character_list.append(j[0])
   

('6', ['OFFICER', 'Dis-mount!', 117.283, 118.716])
('8', ['FIRST SCOUT', "Herman's horsesick!", 124.991, 127.425])
('9', ['SCOUTMASTER', "Chaps, don't anybody wander off. Some of the passageways in here can run for miles.", 137.137, 140.3])
('11', ['HERMAN', "I don't think this is such a good idea.", 150.617, 153.711])
('11', ['HERMAN', 'What is it?', 157.19, 158.885])
('11', ['FEDORA', 'Alfred, did you get anything yet?', 184.05, 185.642])
('11', ['MAN', 'Nothing. Dig in.', 185.718, 187.708])
('11', ['ROSCOE', "Hey, I've got something!", 187.787, 188.754])
('11', ['ROSCOE', 'I got something, Garth!', 191.558, 192.547])
('11', ['ROSCOE', 'I got something... I got something right here.', 192.625, 195.185])
('11', ['ROSCOE', "(more whoops; then) Oh, look at that! (more whoops) We're rich! We're rich!", 201.801, 205.571])
('11', ['HALFBREED', 'Shut up. Shut up.', 205.638, 207.435])
('11', ['HERMAN', 'Indy? Indy? What are they doing? Indiana? Indiana?', 207.507, 219.107])
('11', ['INDY', '

In [52]:
graph_matrix = np.zeros((len(character_list), len(character_list)))
current_scene = 0

for s, d in scene_dialogues:
    if s != current_scene:
        current_scene = s
    
    #work out a way to calculate weights for each scene individually, sum of total time for each character?

SyntaxError: ignored